In [1]:
#!/usr/bin/env python3
"""
Head-to-Head Comparison: Equivariant vs Non-Equivariant Models

This notebook cell version:
- Allows interactive execution without command-line arguments
- Preserves all functionality of compare_models.py
"""

import sys
import argparse
from pathlib import Path
import numpy as np
import pickle
import json
import time
from typing import Dict, Tuple, Any, List
from dataclasses import dataclass, asdict

# Add mmml to path (adjust if necessary)
repo_root = Path(".").resolve().parent.parent.parent
sys.path.insert(0, str(repo_root))

import jax
import jax.numpy as jnp
from scipy.spatial.transform import Rotation

# Import training script components
from trainer import (
    JointPhysNetDCMNet,
    JointPhysNetNonEquivariant,
    load_combined_data,
    train_model,
    create_optimizer,
    get_recommended_optimizer_config,
    LossTerm,
)

try:
    import matplotlib.pyplot as plt
    import matplotlib
    matplotlib.use('Agg')
    HAS_MATPLOTLIB = True
except ImportError:
    HAS_MATPLOTLIB = False


@dataclass
class ModelMetrics:
    """Store metrics for a single model."""
    name: str
    training_time: float
    inference_time: float
    memory_usage_mb: float
    num_parameters: int
    val_energy_mae: float
    val_forces_mae: float
    val_dipole_mae: float
    val_esp_mae: float
    rotation_error_dipole: float
    rotation_error_esp: float
    translation_error_dipole: float
    translation_error_esp: float


def apply_rotation(positions: np.ndarray, rotation_matrix: np.ndarray) -> np.ndarray:
    return np.einsum('ij,snj->sni', rotation_matrix, positions)


def apply_translation(positions: np.ndarray, translation: np.ndarray) -> np.ndarray:
    return positions + translation[None, None, :]


def generate_random_rotation() -> np.ndarray:
    return Rotation.random().as_matrix()


def predict_single(model: Any, params: Any, R: np.ndarray, Z: np.ndarray, N: np.ndarray, vdw_surface: np.ndarray) -> Dict[str, np.ndarray]:
    import e3x

    natoms = R.shape[1]
    batch_size = 1
    positions_flat = R.reshape(-1, 3)
    atomic_numbers_flat = Z.reshape(-1)

    cutoff = 10.0
    n_atoms = int(N[0])
    dst_idx_list = []
    src_idx_list = []
    for i in range(n_atoms):
        for j in range(n_atoms):
            if i != j:
                dist = np.linalg.norm(R[0, i] - R[0, j])
                if dist < cutoff:
                    dst_idx_list.append(i)
                    src_idx_list.append(j)

    dst_idx = jnp.array(dst_idx_list, dtype=jnp.int32)
    src_idx = jnp.array(src_idx_list, dtype=jnp.int32)

    batch_segments = jnp.zeros(natoms, dtype=jnp.int32)
    batch_mask = jnp.ones(batch_size)
    atom_mask = (jnp.arange(natoms) < n_atoms).astype(jnp.float32)

    output = model.apply(
        params,
        atomic_numbers=jnp.array(atomic_numbers_flat),
        positions=jnp.array(positions_flat),
        dst_idx=dst_idx,
        src_idx=src_idx,
        batch_segments=batch_segments,
        batch_size=batch_size,
        batch_mask=batch_mask,
        atom_mask=atom_mask,
    )

    mono_dist = output['mono_dist'].reshape(batch_size, natoms, -1)
    dipo_dist = output['dipo_dist'].reshape(batch_size, natoms, -1, 3)

    esp_pred = calculate_esp(
        mono_dist[0],
        dipo_dist[0],
        vdw_surface[0],
        atom_mask,
    )

    return {
        'dipole': np.array(output['dipoles_dcmnet'][0]),
        'esp': np.array(esp_pred),
        'energy': np.array(output['energy'][0]),
    }


def calculate_esp(charges: jnp.ndarray, positions: jnp.ndarray, grid_points: jnp.ndarray, atom_mask: jnp.ndarray) -> jnp.ndarray:
    natoms, n_dcm = charges.shape
    ngrid = grid_points.shape[0]

    charges_flat = charges.reshape(-1)
    positions_flat = positions.reshape(-1, 3)
    atom_mask_expanded = jnp.repeat(atom_mask, n_dcm)
    charges_masked = charges_flat * atom_mask_expanded

    diff = grid_points[:, None, :] - positions_flat[None, :, :]
    distances = jnp.linalg.norm(diff, axis=-1)
    distances = jnp.where(distances < 1e-6, 1e6, distances)

    distances_bohr = distances * 1.88973
    esp = jnp.sum(charges_masked[None, :] / distances_bohr, axis=1)
    return esp


def test_equivariance(model, params, test_data, num_test_samples=10, seed=42):
    rng = np.random.RandomState(seed)
    n_available = len(test_data['E'])
    test_indices = rng.choice(n_available, size=min(num_test_samples, n_available), replace=False)

    rotation_errors_dipole = []
    rotation_errors_esp = []
    translation_errors_dipole = []
    translation_errors_esp = []

    for idx in test_indices:
        R = test_data['R'][idx:idx+1]
        Z = test_data['Z'][idx:idx+1]
        N = test_data['N'][idx:idx+1]
        vdw_surface = test_data['vdw_surface'][idx:idx+1]

        output_orig = predict_single(model, params, R, Z, N, vdw_surface)

        rot_matrix = generate_random_rotation()
        R_rot = apply_rotation(R, rot_matrix)
        vdw_rot = apply_rotation(vdw_surface, rot_matrix)
        output_rot = predict_single(model, params, R_rot, Z, N, vdw_rot)

        dipole_orig = output_orig['dipole']
        dipole_rot = output_rot['dipole']
        dipole_expected = np.dot(rot_matrix, dipole_orig)
        rotation_error_dipole = np.linalg.norm(dipole_rot - dipole_expected)
        rotation_errors_dipole.append(rotation_error_dipole)

        esp_orig = output_orig['esp']
        esp_rot = output_rot['esp']
        rotation_error_esp = np.mean(np.abs(esp_rot - esp_orig))
        rotation_errors_esp.append(rotation_error_esp)

        translation = rng.randn(3) * 5.0
        R_trans = apply_translation(R, translation)
        vdw_trans = apply_translation(vdw_surface, translation)
        output_trans = predict_single(model, params, R_trans, Z, N, vdw_trans)

        dipole_trans = output_trans['dipole']
        translation_error_dipole = np.linalg.norm(dipole_trans - dipole_orig)
        translation_errors_dipole.append(translation_error_dipole)

        esp_trans = output_trans['esp']
        translation_error_esp = np.mean(np.abs(esp_trans - esp_orig))
        translation_errors_esp.append(translation_error_esp)

    return {
        'rotation_error_dipole': float(np.mean(rotation_errors_dipole)),
        'rotation_error_esp': float(np.mean(rotation_errors_esp)),
        'translation_error_dipole': float(np.mean(translation_errors_dipole)),
        'translation_error_esp': float(np.mean(translation_errors_esp)),
    }


def measure_inference_time(model, params, test_data, num_samples=50):
    indices = np.random.choice(len(test_data['E']), size=min(num_samples, len(test_data['E'])), replace=False)
    times = []
    for idx in indices:
        R = test_data['R'][idx:idx+1]
        Z = test_data['Z'][idx:idx+1]
        N = test_data['N'][idx:idx+1]
        vdw_surface = test_data['vdw_surface'][idx:idx+1]
        start = time.time()
        _ = predict_single(model, params, R, Z, N, vdw_surface)
        times.append(time.time() - start)
    return float(np.mean(times))


def count_parameters(params):
    return sum(x.size for x in jax.tree_util.tree_leaves(params))


# ========== Example interactive setup for notebook ==========

class Args:
    train_efd = Path("/scicore/home/meuwly/boitti0000/mmml/examples/co2/preclassified_data/energies_forces_dipoles_train.npz")
    train_esp = Path("/scicore/home/meuwly/boitti0000/mmml/examples/co2/preclassified_data/grids_esp_train.npz")
    valid_efd = Path("/scicore/home/meuwly/boitti0000/mmml/examples/co2/preclassified_data/energies_forces_dipoles_valid.npz")
    valid_esp = Path("/scicore/home/meuwly/boitti0000/mmml/examples/co2/preclassified_data/grids_esp_valid.npz")
    epochs = 500
    batch_size = 100
    seed = 42
    physnet_features = 64
    dcmnet_features = 128
    n_dcm = 3
    comparison_name = "interactive_comparison"
    output_dir = Path("comparisons")
    skip_training = True
    equivariance_samples = 10

args = Args()
print("Notebook arguments initialized. Ready to run comparison interactively.")


Notebook arguments initialized. Ready to run comparison interactively.


In [2]:
import jax
jax.devices()

[CudaDevice(id=0)]

In [3]:
args

In [4]:
from compare_models import *

In [7]:

# Create output directory
output_dir = args.output_dir / args.comparison_name
output_dir.mkdir(exist_ok=True, parents=True)

print("="*70)
print("HEAD-TO-HEAD MODEL COMPARISON")
print("="*70)
print(f"\nComparison: {args.comparison_name}")
print(f"Output directory: {output_dir}")

# Load data
print(f"\n{'#'*70}")
print("# Loading Data")
print(f"{'#'*70}\n")

train_data = load_combined_data(args.train_efd, args.train_esp, verbose=True)
valid_data = load_combined_data(args.valid_efd, args.valid_esp, verbose=True)

print(f"\n✅ Data loaded:")
print(f"  Training samples: {len(train_data['E'])}")
print(f"  Validation samples: {len(valid_data['E'])}")

# Get dataset properties
natoms = train_data['R'].shape[1]
max_atomic_number = int(max(np.max(train_data['Z']), np.max(valid_data['Z'])))

# Shared PhysNet config
physnet_config = {
    'features': args.physnet_features,
    'max_degree': 0,
    'num_iterations': 3,
    'num_basis_functions': 64,
    'cutoff': 6.0,
    'max_atomic_number': max_atomic_number,
    'charges': True,
    'natoms': natoms,
    'total_charge': 0.0,
    'n_res': 3,
    'zbl': False,
    'use_energy_bias': True,
    'debug': False,
    'efa': False,
}

# Get recommended optimizer settings
recommended_config = get_recommended_optimizer_config(
    dataset_size=len(train_data['E']),
    num_features=args.physnet_features + args.dcmnet_features,
    num_atoms=natoms,
    optimizer_name='adamw',
)

# Model configurations
dcmnet_config = {
    'features': args.dcmnet_features,
    'max_degree': 2,
    'num_iterations': 2,
    'num_basis_functions': 64,
    'cutoff': 10.0,
    'max_atomic_number': max_atomic_number,
    'n_dcm': args.n_dcm,
    'include_pseudotensors': False,
}

noneq_config = {
    'features': args.dcmnet_features * (6 * 3) ,  
    'n_dcm': args.n_dcm,
    'max_atomic_number': max_atomic_number,
    'num_layers': 5,
    'max_displacement': 1.0,
}

results = {}

# ==================================================================
# Train/Load DCMNet (Equivariant)
# ==================================================================
print(f"\n{'#'*70}")
print("# DCMNet (Equivariant) Model")
print(f"{'#'*70}\n")

dcm_ckpt_dir = output_dir / 'dcmnet_equivariant'
dcm_ckpt_dir.mkdir(exist_ok=True, parents=True)

model_dcm = JointPhysNetDCMNet(
    physnet_config=physnet_config,
    dcmnet_config=dcmnet_config,
    mix_coulomb_energy=False,
)

if args.skip_training and (dcm_ckpt_dir / 'best_params.pkl').exists():
    print(f"⏩ Loading existing checkpoint: {dcm_ckpt_dir / 'best_params.pkl'}")
    with open(dcm_ckpt_dir / 'best_params.pkl', 'rb') as f:
        params_dcm = pickle.load(f)
    training_time_dcm = 0.0
else:
    print("🏋️  Training DCMNet...")
    start_time = time.time()
    
    # Create loss terms for training
    dipole_terms = (
        LossTerm(source='physnet', weight=25.0, metric='l2', name='physnet'),
    )
    esp_terms = (
        LossTerm(source='dcmnet', weight=10000.0, metric='l2', name='dcmnet'),
    )
    
    params_dcm = train_model(
        model=model_dcm,
        train_data=train_data,
        valid_data=valid_data,
        num_epochs=args.epochs,
        batch_size=args.batch_size,
        learning_rate=recommended_config['learning_rate'],
        weight_decay=recommended_config['weight_decay'],
        energy_w=10.0,
        forces_w=50.0,
        dipole_w=25.0,
        esp_w=10000.0,
        mono_w=100.0,
        n_dcm=args.n_dcm,
        cutoff=10.0,
        seed=args.seed,
        ckpt_dir=dcm_ckpt_dir.parent,
        name=dcm_ckpt_dir.name,
        print_freq=5,
        dipole_terms=dipole_terms,
        esp_terms=esp_terms,
        optimizer_name='adamw',
        optimizer_kwargs={'b1': 0.9, 'b2': 0.999},
    )
    
    training_time_dcm = time.time() - start_time
    print(f"\n✅ DCMNet training completed in {training_time_dcm/3600:.2f} hours")

# ==================================================================
# Train/Load Non-Equivariant
# ==================================================================
print(f"\n{'#'*70}")
print("# Non-Equivariant Model")
print(f"{'#'*70}\n")

noneq_ckpt_dir = output_dir / 'noneq_model'
noneq_ckpt_dir.mkdir(exist_ok=True, parents=True)

model_noneq = JointPhysNetNonEquivariant(
    physnet_config=physnet_config,
    noneq_config=noneq_config,
    mix_coulomb_energy=False,
)

if args.skip_training and (noneq_ckpt_dir / 'best_params.pkl').exists():
    print(f"⏩ Loading existing checkpoint: {noneq_ckpt_dir / 'best_params.pkl'}")
    with open(noneq_ckpt_dir / 'best_params.pkl', 'rb') as f:
        params_noneq = pickle.load(f)
    training_time_noneq = 0.0
else:
    print("🏋️  Training Non-Equivariant model...")
    start_time = time.time()
    
    # Create loss terms for training (same as DCMNet for fair comparison)
    dipole_terms = (
        LossTerm(source='physnet', weight=25.0, metric='l2', name='physnet'),
    )
    esp_terms = (
        LossTerm(source='dcmnet', weight=10000.0, metric='l2', name='dcmnet'),
    )
    
    params_noneq = train_model(
        model=model_noneq,
        train_data=train_data,
        valid_data=valid_data,
        num_epochs=args.epochs,
        batch_size=args.batch_size,
        learning_rate=recommended_config['learning_rate'],
        weight_decay=recommended_config['weight_decay'],
        energy_w=10.0,
        forces_w=50.0,
        dipole_w=25.0,
        esp_w=10000.0,
        mono_w=100.0,
        n_dcm=args.n_dcm,
        cutoff=10.0,
        seed=args.seed,
        ckpt_dir=noneq_ckpt_dir.parent,
        name=noneq_ckpt_dir.name,
        print_freq=5,
        dipole_terms=dipole_terms,
        esp_terms=esp_terms,
        optimizer_name='adamw',
        optimizer_kwargs={'b1': 0.9, 'b2': 0.999},
    )
    
    training_time_noneq = time.time() - start_time
    print(f"\n✅ Non-Equivariant training completed in {training_time_noneq/3600:.2f} hours")

# ==================================================================
# Run Equivariance Tests
# ==================================================================
print(f"\n{'#'*70}")
print("# Equivariance Testing")
print(f"{'#'*70}")

print("\n--- DCMNet (Equivariant) ---")
equivariance_dcm = test_equivariance(
    model_dcm,
    params_dcm,
    valid_data,
    num_test_samples=args.equivariance_samples,
    seed=args.seed,
)

print("\n--- Non-Equivariant ---")
equivariance_noneq = test_equivariance(
    model_noneq,
    params_noneq,
    valid_data,
    num_test_samples=args.equivariance_samples,
    seed=args.seed,
)

# ==================================================================
# Measure Performance
# ==================================================================
print(f"\n{'#'*70}")
print("# Performance Benchmarking")
print(f"{'#'*70}\n")

# print("Measuring inference times...")
# inference_time_dcm = measure_inference_time(model_dcm, params_dcm, valid_data, num_samples=50)
# inference_time_noneq = measure_inference_time(model_noneq, params_noneq, valid_data, num_samples=50)

# print(f"  DCMNet:        {inference_time_dcm*1000:.2f} ms/sample")
# print(f"  Non-Eq:        {inference_time_noneq*1000:.2f} ms/sample")
# print(f"  Speedup:       {inference_time_dcm/inference_time_noneq:.2f}×")

num_params_dcm = count_parameters(params_dcm)
num_params_noneq = count_parameters(params_noneq)

print(f"\nParameter counts:")
print(f"  DCMNet:        {num_params_dcm:,} parameters")
print(f"  Non-Eq:        {num_params_noneq:,} parameters")
print(f"  Reduction:     {(1 - num_params_noneq/num_params_dcm)*100:.1f}%")

# Load validation metrics from history (with fallback for missing files)
history_dcm_path = dcm_ckpt_dir / 'history.json'
history_noneq_path = noneq_ckpt_dir / 'history.json'

if history_dcm_path.exists():
    with open(history_dcm_path, 'r') as f:
        history_dcm = json.load(f)
    best_epoch_dcm = np.argmin(history_dcm['val_loss'])
    val_energy_mae_dcm = history_dcm['val_energy_mae'][best_epoch_dcm]
    val_forces_mae_dcm = history_dcm['val_forces_mae'][best_epoch_dcm]
    val_dipole_mae_dcm = history_dcm['val_dipole_mae'][best_epoch_dcm]
    val_esp_mae_dcm = history_dcm['val_esp_mae'][best_epoch_dcm]
else:
    print(f"\n⚠️  Warning: No history file found for DCMNet at {history_dcm_path}")
    print("   Using placeholder values (0.0) for validation metrics.")
    print("   Rerun comparison to get full metrics.")
    val_energy_mae_dcm = 0.0
    val_forces_mae_dcm = 0.0
    val_dipole_mae_dcm = 0.0
    val_esp_mae_dcm = 0.0

if history_noneq_path.exists():
    with open(history_noneq_path, 'r') as f:
        history_noneq = json.load(f)
    best_epoch_noneq = np.argmin(history_noneq['val_loss'])
    val_energy_mae_noneq = history_noneq['val_energy_mae'][best_epoch_noneq]
    val_forces_mae_noneq = history_noneq['val_forces_mae'][best_epoch_noneq]
    val_dipole_mae_noneq = history_noneq['val_dipole_mae'][best_epoch_noneq]
    val_esp_mae_noneq = history_noneq['val_esp_mae'][best_epoch_noneq]
else:
    print(f"\n⚠️  Warning: No history file found for Non-Eq at {history_noneq_path}")
    print("   Using placeholder values (0.0) for validation metrics.")
    print("   Rerun comparison to get full metrics.")
    val_energy_mae_noneq = 0.0
    val_forces_mae_noneq = 0.0
    val_dipole_mae_noneq = 0.0
    val_esp_mae_noneq = 0.0

# Create metrics objects
metrics_dcm = ModelMetrics(
    name='DCMNet (Equivariant)',
    training_time=0,
    inference_time=0,
    memory_usage_mb=0.0,  # Would need profiling
    num_parameters=num_params_dcm,
    val_energy_mae=val_energy_mae_dcm,
    val_forces_mae=val_forces_mae_dcm,
    val_dipole_mae=val_dipole_mae_dcm,
    val_esp_mae=val_esp_mae_dcm,
    rotation_error_dipole=equivariance_dcm['rotation_error_dipole'],
    rotation_error_esp=equivariance_dcm['rotation_error_esp'],
    translation_error_dipole=equivariance_dcm['translation_error_dipole'],
    translation_error_esp=equivariance_dcm['translation_error_esp'],
)

metrics_noneq = ModelMetrics(
    name='Non-Equivariant',
    # training_time=training_time_noneq,
    # inference_time=inference_time_noneq,
    training_time=0,
    inference_time=0,
    memory_usage_mb=0.0,
    num_parameters=num_params_noneq,
    val_energy_mae=val_energy_mae_noneq,
    val_forces_mae=val_forces_mae_noneq,
    val_dipole_mae=val_dipole_mae_noneq,
    val_esp_mae=val_esp_mae_noneq,
    rotation_error_dipole=equivariance_noneq['rotation_error_dipole'],
    rotation_error_esp=equivariance_noneq['rotation_error_esp'],
    translation_error_dipole=equivariance_noneq['translation_error_dipole'],
    translation_error_esp=equivariance_noneq['translation_error_esp'],
)

# ==================================================================
# Generate Comparison Report
# ==================================================================
print(f"\n{'#'*70}")
print("# Final Comparison Report")
print(f"{'#'*70}\n")

print("VALIDATION PERFORMANCE:")
print(f"  Energy MAE:  DCMNet={metrics_dcm.val_energy_mae:.4f}, Non-Eq={metrics_noneq.val_energy_mae:.4f}")
print(f"  Forces MAE:  DCMNet={metrics_dcm.val_forces_mae:.4f}, Non-Eq={metrics_noneq.val_forces_mae:.4f}")
print(f"  Dipole MAE:  DCMNet={metrics_dcm.val_dipole_mae:.4f}, Non-Eq={metrics_noneq.val_dipole_mae:.4f}")
print(f"  ESP MAE:     DCMNet={metrics_dcm.val_esp_mae:.4f}, Non-Eq={metrics_noneq.val_esp_mae:.4f}")

print("\nEQUIVARIANCE (Rotation):")
print(f"  Dipole:      DCMNet={metrics_dcm.rotation_error_dipole:.6f}, Non-Eq={metrics_noneq.rotation_error_dipole:.6f}")
print(f"  ESP:         DCMNet={metrics_dcm.rotation_error_esp:.6f}, Non-Eq={metrics_noneq.rotation_error_esp:.6f}")
print("  ⚠️  DCMNet should have near-zero rotation error (equivariant)")
print("  ⚠️  Non-Eq will have larger error (not equivariant)")

print("\nINVARIANCE (Translation):")
print(f"  Dipole:      DCMNet={metrics_dcm.translation_error_dipole:.6f}, Non-Eq={metrics_noneq.translation_error_dipole:.6f}")
print(f"  ESP:         DCMNet={metrics_dcm.translation_error_esp:.6f}, Non-Eq={metrics_noneq.translation_error_esp:.6f}")
print("  ✅ Both should have near-zero translation error")

# print("\nCOMPUTATIONAL EFFICIENCY:")
# if training_time_dcm > 0 and training_time_noneq > 0:
#     speedup = training_time_dcm / training_time_noneq
#     print(f"  Training:    DCMNet={training_time_dcm/3600:.2f}h, Non-Eq={training_time_noneq/3600:.2f}h ({speedup:.2f}× speedup)")
# print(f"  Inference:   DCMNet={inference_time_dcm*1000:.2f}ms, Non-Eq={inference_time_noneq*1000:.2f}ms ({inference_time_dcm/inference_time_noneq:.2f}× speedup)")
# print(f"  Parameters:  DCMNet={num_params_dcm:,}, Non-Eq={num_params_noneq:,} ({(1-num_params_noneq/num_params_dcm)*100:.1f}% reduction)")

# Save results
# Convert Path objects to strings for JSON serialization
args_dict = {}
for key, value in vars(args).items():
    if isinstance(value, Path):
        args_dict[key] = str(value)
    else:
        args_dict[key] = value

results = {
    'dcmnet': asdict(metrics_dcm),
    'noneq': asdict(metrics_noneq),
    'args': args_dict,
}

with open(output_dir / 'comparison_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print(f"\n✅ Results saved to: {output_dir / 'comparison_results.json'}")

# Generate plots
print(f"\n{'#'*70}")
print("# Generating Comparison Plots")
print(f"{'#'*70}\n")

plot_comparison(metrics_dcm, metrics_noneq, output_dir)

print(f"\n{'='*70}")
print("✅ COMPARISON COMPLETE!")
print(f"{'='*70}")
print(f"\nAll results saved to: {output_dir}")
print("\nGenerated files:")
print(f"  - comparison_results.json")
print(f"  - performance_comparison.png")
print(f"  - efficiency_comparison.png")
print(f"  - equivariance_comparison.png")
print(f"  - dcmnet_equivariant/ (checkpoint)")
print(f"  - noneq_model/ (checkpoint)")


HEAD-TO-HEAD MODEL COMPARISON

Comparison: interactive_comparison
Output directory: comparisons/interactive_comparison

######################################################################
# Loading Data
######################################################################

  Loading EFD: /scicore/home/meuwly/boitti0000/mmml/examples/co2/preclassified_data/energies_forces_dipoles_train.npz
  Loading ESP: /scicore/home/meuwly/boitti0000/mmml/examples/co2/preclassified_data/grids_esp_train.npz
  ✅ Subtracted atomic energies (now relative to isolated atoms)
  Aligning ESP grids to molecular reference frames...
  ✅ Aligned ESP grids to molecular reference frames
     Sample 0 - Atom COM: [0.         0.14525378 0.11470596]
     Sample 0 - Grid COM before: [3.25342429 3.24367862 3.3973045 ]
     Sample 0 - Grid COM after: [2.56091444e-16 1.45253775e-01 1.14705956e-01]
     Sample 0 - Offset corrected: [3.25342429 3.09842484 3.28259855] Å
  Combined data shapes:
    R: (8000, 60, 3)
    Z:

In [21]:
test_efd = Path("/scicore/home/meuwly/boitti0000/mmml/examples/co2/preclassified_data/energies_forces_dipoles_test.npz")
test_esp = Path("/scicore/home/meuwly/boitti0000/mmml/examples/co2/preclassified_data/grids_esp_test.npz")

ERROR! Session/line number was not unique in database. History logging moved to new session 165


In [23]:
test_data = load_combined_data(test_efd, test_esp, verbose=True)

  Loading EFD: /scicore/home/meuwly/boitti0000/mmml/examples/co2/preclassified_data/energies_forces_dipoles_test.npz
  Loading ESP: /scicore/home/meuwly/boitti0000/mmml/examples/co2/preclassified_data/grids_esp_test.npz
  ✅ Subtracted atomic energies (now relative to isolated atoms)
  Aligning ESP grids to molecular reference frames...
  ✅ Aligned ESP grids to molecular reference frames
     Sample 0 - Atom COM: [0.         0.1790964  0.06964373]
     Sample 0 - Grid COM before: [3.25245414 3.3157224  3.30973476]
     Sample 0 - Grid COM after: [-2.26877776e-15  1.79096404e-01  6.96437267e-02]
     Sample 0 - Offset corrected: [3.25245414 3.13662599 3.24009103] Å
  Combined data shapes:
    R: (1000, 60, 3)
    Z: (1000, 60)
    N: (1000,)
    E: (1000,)
    F: (1000, 60, 3)
    Dxyz: (1000, 3)
    esp: (1000, 3000)
    vdw_surface: (1000, 3000, 3)
  Data padding: 60 atoms
  Max actual atoms: 3


In [24]:
print(f"\n{'#'*70}")
print("# Equivariance Testing")
print(f"{'#'*70}")
NTEST = 10
print("\n--- DCMNet (Equivariant) ---")
equivariance_dcm = test_equivariance(
    model_dcm,
    params_dcm,
    test_data,
    num_test_samples=NTEST,
    seed=args.seed,
)

print("\n--- Non-Equivariant ---")
equivariance_noneq = test_equivariance(
    model_noneq,
    params_noneq,
    test_data,
    num_test_samples=NTEST,
    seed=args.seed,
)



######################################################################
# Equivariance Testing
######################################################################

--- DCMNet (Equivariant) ---

Testing Equivariance on 10 samples

Rotation Equivariance:
  Dipole error: 0.000253 ± 0.000148 e·Å
  ESP error:    0.000054 ± 0.000022 Ha/e

Translation Invariance:
  Dipole error: 0.000000 ± 0.000000 e·Å
  ESP error:    0.000000 ± 0.000000 Ha/e

--- Non-Equivariant ---

Testing Equivariance on 10 samples

Rotation Equivariance:
  Dipole error: 0.067412 ± 0.063540 e·Å
  ESP error:    0.004880 ± 0.001656 Ha/e

Translation Invariance:
  Dipole error: 0.000045 ± 0.000063 e·Å
  ESP error:    0.000007 ± 0.000010 Ha/e


In [25]:
equivariance_noneq

{'rotation_error_dipole': 0.06741162832557471,
 'rotation_error_esp': 0.004880109336227179,
 'translation_error_dipole': 4.460750278667547e-05,
 'translation_error_esp': 6.7671594479179475e-06,
 'rotation_error_dipole_std': 0.0635396414212656,
 'rotation_error_esp_std': 0.0016557491617277265,
 'translation_error_dipole_std': 6.276926433201879e-05,
 'translation_error_esp_std': 9.501078238827176e-06}

In [26]:
equivariance_dcm

{'rotation_error_dipole': 0.0002530550989751969,
 'rotation_error_esp': 5.352227162802592e-05,
 'translation_error_dipole': 2.7987181283606333e-07,
 'translation_error_esp': 1.4460646013958467e-08,
 'rotation_error_dipole_std': 0.0001480252382412301,
 'rotation_error_esp_std': 2.1615032892441377e-05,
 'translation_error_dipole_std': 3.7980612432875205e-07,
 'translation_error_esp_std': 1.2076434963148586e-08}